In [4]:
#!pip install pycaret
!pip install mysql-connector-python
!pip install sqlalchemy==1.4.32

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/99/72/9bcbc7d6fe122fcbb4f948fd79c18387e84c4df8762aa55eb230acf1193b/mysql_connector_python-8.1.0-py2.py3-none-any.whl.metadata
  Using cached protobuf-4.21.12-cp37-abi3-macosx_10_9_universal2.whl (486 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.1/581.1 kB 6.4 MB/s eta 0:00:0000:0100:01
  Using cached SQLAlchemy-1.4.32.tar.gz (8.1 MB)
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.4.32-cp38-cp38-macosx_11_0_arm64.whl size=1535159 sha256=d152dac4f0825b5ccde4b649951cb7c8138ef7a639bfc7fe8f85c647c74949b5
  Stored in directory: /Users/martinmfejka/Library/Caches/pip/wheels/f1/89/a6/de94839bd1eae1e97da3e026f85938bc8afca6739bda1df206
Successfully built sqlalchemy


In [61]:
import sklearn as sk
import pycaret
import pandas as pd
import numpy as np
import sqlalchemy
import mysql.connector
from constants import API_KEY, database_name, sql_hostname

from pycaret.classification import *

from datetime import datetime
#from google.colab import drive
# from google.colab.data_table import DataTable
# from google.colab import files

In [66]:
engine = sqlalchemy.create_engine(f'{sql_hostname}/{database_name}')

Raw_Dataset = pd.read_sql(sql = "SELECT * FROM baseball_spread", con = engine).set_index("game_datetime").reset_index()[["game_datetime", "team_1",	"team_1_spread_odds",	"team_2",	"team_2_spread_odds",	"venue_name",	"spread"]]
print(len(Raw_Dataset))
#Raw_Dataset['game_datetime'] = pd.to_numeric(pd.to_datetime(Raw_Dataset['game_datetime']))

768


In [67]:
Raw_Dataset

,game_datetime,team_1,team_1_spread_odds,team_2,team_2_spread_odds,venue_name,spread
0,2023-04-04 23:05:00,Tampa Bay Rays,135,Washington Nationals,-165,Nationals Park,1
1,2023-04-04 23:40:00,Toronto Blue Jays,-195,Kansas City Royals,160,Kauffman Stadium,1
2,2023-04-04 23:40:00,New York Mets,155,Milwaukee Brewers,-190,American Family Field,0
3,2023-04-05 17:05:00,Tampa Bay Rays,110,Washington Nationals,-140,Nationals Park,1
4,2023-04-05 19:37:00,Cleveland Guardians,160,Oakland Athletics,-195,Oakland Coliseum,1
...,...,...,...,...,...,...,...
763,2023-08-10 22:40:00,Tampa Bay Rays,105,St. Louis Cardinals,-125,Tropicana Field,0
764,2023-08-10 22:40:00,Philadelphia Phillies,-122,Washington Nationals,102,Citizens Bank Park,1
765,2023-08-10 23:10:00,Boston Red Sox,-125,Kansas City Royals,105,Fenway Park,1
766,2023-08-11 02:10:00,Los Angeles Dodgers,-166,Colorado Rockies,140,Dodger Stadium,0


In [68]:
Raw_Dataset.sort_values('game_datetime', ascending=False)

,game_datetime,team_1,team_1_spread_odds,team_2,team_2_spread_odds,venue_name,spread
767,2023-08-11 22:40:00,Tampa Bay Rays,120,Cleveland Guardians,-142,Tropicana Field,0
766,2023-08-11 02:10:00,Los Angeles Dodgers,-166,Colorado Rockies,140,Dodger Stadium,0
765,2023-08-10 23:10:00,Boston Red Sox,-125,Kansas City Royals,105,Fenway Park,1
764,2023-08-10 22:40:00,Philadelphia Phillies,-122,Washington Nationals,102,Citizens Bank Park,1
763,2023-08-10 22:40:00,Tampa Bay Rays,105,St. Louis Cardinals,-125,Tropicana Field,0
...,...,...,...,...,...,...,...
1,2023-04-04 23:40:00,Toronto Blue Jays,-195,Kansas City Royals,160,Kauffman Stadium,1
284,2023-04-04 23:10:00,Boston Red Sox,140,Pittsburgh Pirates,-175,Fenway Park,0
283,2023-04-04 23:05:00,New York Yankees,110,Philadelphia Phillies,-140,Yankee Stadium,0
0,2023-04-04 23:05:00,Tampa Bay Rays,135,Washington Nationals,-165,Nationals Park,1


Preprocessed_DF = pd.concat((Raw_Dataset[['game_datetime']], pd.get_dummies(Raw_Dataset[['team_1','team_2','venue_name']]), Raw_Dataset[['team_1_spread_odds', 'team_2_spread_odds', 'spread']]), axis=1)

cat_features = Preprocessed_DF.columns.tolist()
cat_features.remove("game_datetime")
Preprocessed_DF['game_datetime'] = pd.to_numeric(pd.to_datetime(Preprocessed_DF['game_datetime']))


In [21]:
Classification_Setup = setup(data = Raw_Dataset, target = "spread", train_size = .70, use_gpu = True, data_split_stratify = False, data_split_shuffle = False, preprocess=True, fold_shuffle=True)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please re

,Description,Value
0,Session id,1942
1,Target,spread
2,Target type,Binary
3,Original data shape,"(768, 7)"
4,Transformed data shape,"(768, 9)"
5,Transformed train set shape,"(537, 9)"
6,Transformed test set shape,"(231, 9)"
7,Numeric features,2
8,Date features,1
9,Categorical features,3


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the prov

[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
[LightGBM] [Fatal] CUDA Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_CUDAP=1



***Compare/Create Model***

In [22]:
Classification_Models = Classification_Setup.compare_models(turbo = True)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
knn,K Neighbors Classifier,0.6779,0.6903,0.8011,0.7063,0.7485,0.3026,0.3107,0.1150
lr,Logistic Regression,0.6703,0.7177,0.7144,0.7309,0.7220,0.3167,0.3176,0.0890
ada,Ada Boost Classifier,0.6668,0.6891,0.7705,0.7048,0.7335,0.2877,0.2932,0.1390
rf,Random Forest Classifier,0.6631,0.6814,0.7548,0.7040,0.7277,0.2857,0.2888,0.2580
ridge,Ridge Classifier,0.6611,0.0000,0.7362,0.7098,0.7218,0.2876,0.2895,0.0630
gbc,Gradient Boosting Classifier,0.6593,0.6858,0.7547,0.7030,0.7258,0.2750,0.2797,0.1630
lda,Linear Discriminant Analysis,0.6555,0.7007,0.7394,0.7021,0.7192,0.2737,0.2760,0.0840
et,Extra Trees Classifier,0.6538,0.6832,0.7520,0.6965,0.7215,0.2638,0.2680,0.2340
nb,Naive Bayes,0.6499,0.6848,0.6242,0.7507,0.6799,0.3003,0.3077,0.0830
lightgbm,Light Gradient Boosting Machine,0.6407,0.6487,0.7241,0.6916,0.7060,0.2433,0.2461,0.2520


You can use the multi-layer perceptron ("mlp") as outlined in the post, but a random forest ("rf") classification is more effective. A neural-network is a ***bit*** of an overkill.

In [58]:
model_name = "nb"
Classification_Model = Classification_Setup.create_model(model_name)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6852,0.6537,0.6667,0.7857,0.7213,0.3651,0.3717
1,0.6481,0.6883,0.5758,0.7917,0.6667,0.3133,0.3313
2,0.6296,0.7145,0.5312,0.7727,0.6296,0.2838,0.3040
3,0.5926,0.6548,0.5312,0.7083,0.6071,0.2016,0.2107
4,0.6852,0.7060,0.7188,0.7419,0.7302,0.3526,0.3529
5,0.6111,0.5980,0.5938,0.7037,0.6441,0.2222,0.2261
6,0.7037,0.7713,0.6875,0.7857,0.7333,0.4033,0.4079
7,0.6038,0.6622,0.6250,0.6897,0.6557,0.1917,0.1930
8,0.6604,0.7009,0.6250,0.7692,0.6897,0.3234,0.3320


In [53]:
Prediction = Classification_Setup.predict_model(Classification_Model)
Prediction.head(20)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.5887,0.6450,0.5507,0.6972,0.6154,0.1864,0.1924


,game_datetime,team_1,team_1_spread_odds,team_2,team_2_spread_odds,venue_name,spread,prediction_label,prediction_score
537,2023-06-06 01:10:00,San Diego Padres,-105,Chicago Cubs,-125,Petco Park,1,0,0.5217
538,2023-06-06 22:40:00,Tampa Bay Rays,145,Minnesota Twins,-180,Tropicana Field,1,1,0.5807
539,2023-06-06 22:40:00,Miami Marlins,-135,Kansas City Royals,105,loanDepot park,1,1,0.9572
540,2023-06-06 23:05:00,New York Yankees,135,Chicago White Sox,-165,Yankee Stadium,0,0,0.7904
541,2023-06-06 23:20:00,Atlanta Braves,105,New York Mets,-135,Truist Park,1,0,0.7028
542,2023-06-07 01:38:00,Los Angeles Angels,130,Chicago Cubs,-160,Angel Stadium,1,0,0.9254
543,2023-06-07 16:35:00,Pittsburgh Pirates,125,Oakland Athletics,-155,PNC Park,0,1,0.9762
544,2023-06-07 20:10:00,San Diego Padres,115,Seattle Mariners,-145,Petco Park,1,0,0.8263
545,2023-06-07 23:10:00,Cleveland Guardians,-130,Boston Red Sox,100,Progressive Field,1,1,0.8284
546,2023-06-07 23:40:00,Milwaukee Brewers,-135,Baltimore Orioles,105,American Family Field,1,1,0.9557


In [59]:
Classification_Setup.evaluate_model(Classification_Model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [60]:
Classification_Model.feature_names_in_

array(['game_datetime_year', 'game_datetime_month', 'game_datetime_day',
       'team_1', 'team_1_spread_odds', 'team_2', 'team_2_spread_odds',
       'venue_name'], dtype=object)

***Save Model***

In [55]:
Finalized_Model = pycaret.classification.finalize_model(Classification_Model)
Finalized_Model_save_to_file_string = f"models/{datetime.today().strftime('%Y-%m-%d')}_Baseball_Spread_model_{model_name}"
pycaret.classification.save_model(Finalized_Model, Finalized_Model_save_to_file_string)
#files.download(f"{Finalized_Model_save_to_file_string}.pkl")

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/var/folders/2c/yw3tyqqx7tn_wgl7j28qh2dw0000gn/T/joblib),
          steps=[('date_feature_extractor',
                  TransformerWrapper(exclude=None, include=['game_datetime'],
                                     transformer=ExtractDateTimeFeatures(features=['day',
                                                                                   'month',
                                                                                   'year']))),
                 ('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['team_1_spread_odds',
                                              'team_2_spre...
                  TransformerWrapper(exclude=None,
                                     include=['team_1', 'team_2', 'venue_name'],
                                     transformer=TargetEncoder(cols=['team_1',
                                                                     'team_